In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from math import sqrt
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from keras import metrics
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive,
                                         drift, 
                                         mean, 
                                         seasonal_naive)

data = pd.read_csv('/Users/alket/Desktop/dati/new_data_backfill_forwfill.csv',index_col = 0)

# aggregazione dati per celle
agg_by_cell = data.groupby(by = ['cell_num'])

In [2]:
trend_data = []
residual_data = []
seasonal_data = []

In [ ]:
for i, k in agg_by_cell:
    
    dates4dec = []
    cell_values = []
    k = k.iloc[::4,:]
    print(len(k))
    for index, row in k.iterrows():
        
        date = row['date']
        h = str(row['hours'])
   
        h = h.split('.')
    
        if len(h[0])<2:
            h = h[1]+h[0]
        else: 
            h = h[0]
   
        minutes = str(row['minutes'])
        m = ''
        minutes = minutes.split('.')
        if len(minutes[0])<2: 
            m = minutes[0] +'0'
        else: 
            m = minutes[0]
        #print(date, h, m)
        data_f = date+' '+h+':'+m+':'+'00'
        #print(data_f)
        cell_values.append(row['nr_people'])
        dates4dec.append(data_f) 
        
    dict_i = {'ds': dates4dec, 'y':cell_values}
    data4deco = pd.DataFrame(dict_i, index=None, columns=None)  
    #print(data4deco)
    data4deco['ds'] = pd.to_datetime(data4deco['ds'])
    data4deco = data4deco.set_index('ds')
    #print(data4deco)
    decomp = decompose(data4deco['y'], period=24)
    #with plt.rc_context():
    #    plt.rc("figure", figsize=(20,8))
    #    decomp.plot()
    #    plt.show()
    trend = decomp.trend.values
    print(trend[:10])
    seasonal = decomp.seasonal.values
    residual = decomp.resid.values
    
        
    trend = trend.reshape((len(trend), 1))
    seasonal = seasonal.reshape((len(seasonal), 1))
    residual = residual.reshape((len(residual), 1))
     
    seasonal = seasonal.astype(int)    
    residual = residual.astype(int)
    trend = trend.astype(int)
    print(trend[:10])
    trend_data.append(trend)
    seasonal_data.append(seasonal)
    residual_data.append(residual)

2952
[36.58638794 36.59382377 36.6012596  36.60869543 36.61613126 36.62356709
 36.63100291 36.63843874 36.64587457 36.6533104 ]
[[36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]
 [36]]
2952
[40.20226828 40.21005418 40.21784008 40.22562598 40.23341188 40.24119778
 40.24898368 40.25676957 40.26455547 40.27234137]
[[40]
 [40]
 [40]
 [40]
 [40]
 [40]
 [40]
 [40]
 [40]
 [40]]
2952
[42.11363646 42.12121945 42.12880243 42.13638542 42.14396841 42.1515514
 42.15913438 42.16671737 42.17430036 42.18188334]
[[42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]]
2952
[42.5161606  42.52357758 42.53099456 42.53841154 42.54582852 42.5532455
 42.56066248 42.56807946 42.57549645 42.58291343]
[[42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]]
2952
[42.22365192 42.2303766  42.23710129 42.24382598 42.25055067 42.25727536
 42.26400004 42.27072473 42.27744942 42.28417411]
[[42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]
 [42]]
2952
[41.73186664 41.73667929 41.74149194 41.7463046  41.

[52.77794182 52.77775186 52.77756191 52.77737196 52.77718201 52.77699205
 52.7768021  52.77661215 52.7764222  52.77623224]
[[52]
 [52]
 [52]
 [52]
 [52]
 [52]
 [52]
 [52]
 [52]
 [52]]
2952
[51.0367739  51.03582211 51.03487033 51.03391855 51.03296676 51.03201498
 51.0310632  51.03011141 51.02915963 51.02820785]
[[51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]
 [51]]
2952
[333.65063461 333.64474841 333.63886221 333.632976   333.6270898
 333.6212036  333.61531739 333.60943119 333.60354498 333.59765878]
[[333]
 [333]
 [333]
 [333]
 [333]
 [333]
 [333]
 [333]
 [333]
 [333]]
2952
[433.13765532 433.13294174 433.12822815 433.12351457 433.11880099
 433.11408741 433.10937382 433.10466024 433.09994666 433.09523308]
[[433]
 [433]
 [433]
 [433]
 [433]
 [433]
 [433]
 [433]
 [433]
 [433]]
2952
[393.55033855 393.54986357 393.5493886  393.54891362 393.54843865
 393.54796367 393.5474887  393.54701372 393.54653875 393.54606377]
[[393]
 [393]
 [393]
 [393]
 [393]
 [393]
 [393]
 [393]
 [393]
 [393]]
2

In [ ]:
final_data_trend = hstack((trend_data))
final_data_seasonal = hstack((seasonal_data))
final_data_residual = hstack((residual_data))

In [ ]:
final_data_residual[4]

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
      # find the end of this pattern
      end_ix = i + n_steps_in
      out_end_ix = end_ix + n_steps_out
      # check if we are beyond the dataset
      if out_end_ix > len(sequences):
          break
      # gather input and output parts of the pattern
      seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
      X.append(seq_x)
      y.append(seq_y)
  return np.array(X), np.array(y)

# choose a number of time steps
n_steps_in, n_steps_out = 24, 12

In [ ]:
# convert into input/output
X_seasonal, y_seasonal = split_sequences(final_data_seasonal, n_steps_in, n_steps_out)
X_trend, y_trend = split_sequences(final_data_trend, n_steps_in, n_steps_out)
X_residual, y_residual = split_sequences(final_data_residual, n_steps_in, n_steps_out)

count = 0
for i in range(len(X_seasonal)):
    count += 1
    print('inizia iterazione ', i)
    print(X_seasonal[i], y_seasonal[i])
    if count > 4: break

# flatten input season
n_input_seasonal = X_seasonal.shape[1] * X_seasonal.shape[2]
X_seasonal = X_seasonal.reshape((X_seasonal.shape[0], n_input_seasonal))
# flatten output
n_output_seasonal = y_seasonal.shape[1] * y_seasonal.shape[2]
y_seasonal = y_seasonal.reshape((y_seasonal.shape[0], n_output_seasonal))


# flatten input trend
n_input_trend = X_trend.shape[1] * X_trend.shape[2]
X_trend = X_trend.reshape((X_trend.shape[0], n_input_trend))
# flatten output
n_output_trend = y_trend.shape[1] * y_trend.shape[2]
y_trend = y_trend.reshape((y_trend.shape[0], n_output_trend))

# flatten input residual
n_input_residual = X_residual.shape[1] * X_residual.shape[2]
X_residual = X_residual.reshape((X_residual.shape[0], n_input_residual))
# flatten output residual
n_output_residual = y_residual.shape[1] * y_residual.shape[2]
y_residual = y_residual.reshape((y_residual.shape[0], n_output_residual))

In [ ]:
split_train_test = int(len(X_trend)*0.75)
train_X_trend, test_X_trend = X_trend[:split_train_test], X_trend[split_train_test:]
train_y_trend, test_y_trend = y_trend[:split_train_test], y_trend[split_train_test:]
print(len(train_X_trend), len(train_y_trend), len(test_X_trend), len(test_y_trend))

train_X_seasonal, test_X_seasonal = X_seasonal[:split_train_test], X_seasonal[split_train_test:]
train_y_seasonal, test_y_seasonal = y_seasonal[:split_train_test], y_seasonal[split_train_test:]
print(len(train_X_seasonal), len(train_y_seasonal), len(test_X_seasonal), len(test_y_seasonal))

train_X_residual, test_X_residual = X_residual[:split_train_test], X_residual[split_train_test:]
train_y_residual, test_y_residual = y_residual[:split_train_test], y_residual[split_train_test:]
print(len(train_X_residual), len(train_y_residual), len(test_X_residual), len(test_y_residual))

In [ ]:
# define model
model_trend = Sequential()
model_trend.add(Dense(100, activation='relu', input_dim=n_input_trend)) 
model_trend.add(Dense(50, activation='relu', input_dim=n_input_trend)) 

model_trend.add(Dense(n_output_trend))
model_trend.compile(optimizer='adam', loss='mse',  metrics=[metrics.mae, 'accuracy'])

# fit model
model_trend.fit(train_X_trend, train_y_trend, epochs=100, verbose=2)

In [ ]:
# define model
model_seasonal = Sequential()
model_seasonal.add(Dense(100, activation='relu', input_dim=n_input_seasonal)) 
model_seasonal.add(Dense(50, activation='relu', input_dim=n_input_seasonal))

model_seasonal.add(Dense(n_output_seasonal))
model_seasonal.compile(optimizer='adam', loss='mse',  metrics=[metrics.mae, 'accuracy'])

# fit model
model_seasonal.fit(train_X_seasonal, train_y_seasonal, epochs=100, verbose=2)

In [ ]:
model_residual = Sequential()
model_residual.add(Dense(100, activation='relu', input_dim=n_input_residual)) 
model_residual.add(Dense(50, activation='relu', input_dim=n_input_residual))

model_residual.add(Dense(n_output_residual))
model_residual.compile(optimizer='adam', loss='mse',  metrics=[metrics.mae, 'accuracy'])

# fit model
model_residual.fit(train_X_residual, train_y_residual, epochs=100, verbose=2)

In [ ]:
predicted_trend = np.array([])
for i in range(len(test_X_trend)):
    x_input = test_X_trend[i].reshape((1, n_input_trend))
    yhat = model_trend.predict(x_input, verbose=0)
    #print(yhat[0])
    predicted_trend = np.append(predicted_trend, yhat[0])

In [ ]:
predicted_seasonal = np.array([])
for i in range(len(test_X_seasonal)):
    x_input = test_X_seasonal[i].reshape((1, n_input_seasonal))
    yhat = model_seasonal.predict(x_input, verbose=0)
    #print(yhat[0])
    predicted_seasonal = np.append(predicted_seasonal, yhat[0])
    
predicted_residual = np.array([])    
for i in range(len(test_X_residual)):
    x_input = test_X_residual[i].reshape((1, n_input_trend))
    yhat = model_residual.predict(x_input, verbose=0)
    #print(yhat[0])
    predicted_residual = np.append(predicted_residual, yhat[0])

In [ ]:
predicted_final = predicted_trend + predicted_seasonal + predicted_residual

In [ ]:
predicted_final.shape

In [ ]:
num_data = []
## prepare dataset
for i, k in agg_by_cell:
    cell_i = agg_by_cell.get_group(i)
    cell_i = cell_i.iloc[::4, :]
    # define input sequence
    series_i = cell_i['nr_people'].values
    series_i = series_i.reshape((len(series_i), 1))
    num_data.append(series_i)

In [ ]:
final_data = hstack((num_data))
final_data.shape

In [ ]:
# convert into input/output
X, y = split_sequences(final_data, n_steps_in, n_steps_out)

count = 0
for i in range(len(X)):
    count += 1
    print('inizia iterazione ', i)
    print(X[i], y[i])
    if count > 1: break
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

# flatten output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [ ]:
len(y) * len(y[1])

In [ ]:
split_train_test = int(len(X)*0.75)
train_X, test_X = X[:split_train_test], X[split_train_test:]
train_y, test_y = y[:split_train_test], y[split_train_test:]
print(len(train_X), len(train_y), len(test_X), len(test_y))

In [ ]:
exp_y = test_y.reshape((test_y.shape[0] * test_y.shape[1])) 
len(exp_y)

In [ ]:
difference = abs(predicted_final - exp_y)
print('Mean Absolute Error = ', np.mean(difference))
plt.figure(figsize = (14,2))
plt.boxplot(difference, vert = False)
plt.show()